<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/264_MissionOrchestratorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent selection utilities for Mission Orchestrator Agent

In [ ]:
"""Agent selection utilities for Mission Orchestrator Agent"""

from typing import List, Dict, Any, Optional
from agents.mission_orchestrator.utilities.data_loading import get_agent_by_id, get_capable_agents_for_task


def select_agent_for_task(
    task_id: str,
    capabilities_matrix: List[Dict[str, Any]],
    agents: List[Dict[str, Any]],
    strategy: str = "first_available"
) -> Optional[Dict[str, Any]]:
    """
    Select an agent for a task based on the capabilities matrix.

    MVP: Simple selection - picks first available agent.
    Future: Can add load balancing, skill-based selection, etc.

    Args:
        task_id: Task identifier
        capabilities_matrix: List of task-to-agent mappings
        agents: List of all available agents
        strategy: Selection strategy ("first_available" for MVP)

    Returns:
        Selected agent dictionary, or None if no capable agent found
    """
    # Get capable agent IDs
    capable_agent_ids = get_capable_agents_for_task(task_id)

    if not capable_agent_ids:
        return None

    # MVP: Select first available agent
    if strategy == "first_available":
        first_agent_id = capable_agent_ids[0]
        return get_agent_by_id(first_agent_id)

    # Future strategies can be added here:
    # - "load_balanced": Select agent with least current workload
    # - "skill_based": Select agent with highest skill level
    # - "cost_optimized": Select agent with lowest cost

    return None


def get_capable_agents(
    task_id: str,
    capabilities_matrix: List[Dict[str, Any]],
    agents: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Get all agents capable of performing a task.

    Args:
        task_id: Task identifier
        capabilities_matrix: List of task-to-agent mappings
        agents: List of all available agents

    Returns:
        List of agent dictionaries capable of performing the task
    """
    capable_agent_ids = get_capable_agents_for_task(task_id)
    capable_agents = []

    for agent_id in capable_agent_ids:
        agent = get_agent_by_id(agent_id)
        if agent:
            capable_agents.append(agent)

    return capable_agents



# Task execution utilities for Mission Orchestrator Agent

In [ ]:
"""Task execution utilities for Mission Orchestrator Agent"""

import time
from datetime import datetime
from typing import Dict, Any, Optional


def mock_agent_execution(agent_id: str, task_description: str) -> Dict[str, Any]:
    """
    Mock agent execution for MVP.

    In a real system, this would call the actual agent API/function.
    For MVP, we simulate execution with a simple delay and mock result.

    Args:
        agent_id: Agent identifier
        task_description: Description of the task to execute

    Returns:
        Mock execution result dictionary
    """
    # Simulate execution time (0.1 seconds for testing)
    time.sleep(0.1)

    # Mock result
    result = {
        "status": "completed",
        "agent_id": agent_id,
        "task_description": task_description,
        "output": f"Mock execution result for {task_description} by agent {agent_id}",
        "timestamp": datetime.now().isoformat(),
        "execution_time_seconds": 0.1
    }

    return result


def execute_task(
    task: Dict[str, Any],
    agent: Dict[str, Any],
    mock_execution: bool = True
) -> Dict[str, Any]:
    """
    Execute a task using the specified agent.

    MVP: Uses mock execution. Future: Can call real agent APIs.

    Args:
        task: Task dictionary with task_id, task description, etc.
        agent: Agent dictionary with agent_id, name, etc.
        mock_execution: Whether to use mock execution (True for MVP)

    Returns:
        Execution result dictionary with status, output, timing, etc.
    """
    task_id = task.get("task_id")
    task_description = task.get("task", "")
    agent_id = agent.get("agent_id")
    agent_name = agent.get("name", "")

    start_time = datetime.now()

    try:
        if mock_execution:
            # MVP: Mock execution
            execution_result = mock_agent_execution(agent_id, task_description)
        else:
            # Future: Real agent execution
            # execution_result = call_agent_api(agent_id, task)
            raise NotImplementedError("Real agent execution not implemented yet")

        end_time = datetime.now()
        duration_seconds = (end_time - start_time).total_seconds()
        duration_minutes = duration_seconds / 60.0

        result = {
            "task_id": task_id,
            "task": task_description,
            "agent_id": agent_id,
            "agent_name": agent_name,
            "status": execution_result.get("status", "completed"),
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat(),
            "duration_seconds": duration_seconds,
            "duration_minutes": duration_minutes,
            "result": execution_result,
            "error": None
        }

        return result

    except Exception as e:
        end_time = datetime.now()
        duration_seconds = (end_time - start_time).total_seconds()

        result = {
            "task_id": task_id,
            "task": task_description,
            "agent_id": agent_id,
            "agent_name": agent_name,
            "status": "failed",
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat(),
            "duration_seconds": duration_seconds,
            "duration_minutes": duration_seconds / 60.0,
            "result": None,
            "error": str(e)
        }

        return result



# Task ordering utilities for Mission Orchestrator Agent

In [ ]:
"""Task ordering utilities for Mission Orchestrator Agent"""

from typing import List, Dict, Any, Set, Tuple


def order_tasks_by_dependency(tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Order tasks by their order field (simple sequential ordering).

    Args:
        tasks: List of task dictionaries with 'order' field

    Returns:
        List of tasks sorted by order field
    """
    return sorted(tasks, key=lambda t: t.get("order", 999))


def detect_circular_dependencies(tasks: List[Dict[str, Any]]) -> Tuple[bool, List[str]]:
    """
    Detect circular dependencies in task dependencies.

    Uses DFS to detect cycles in the dependency graph.

    Args:
        tasks: List of task dictionaries with 'task_id' and 'depends_on' fields

    Returns:
        (has_circular_deps, cycle_path): Tuple indicating if cycles exist and path if found
    """
    # Build dependency graph
    graph: Dict[str, List[str]] = {}
    task_ids = set()

    for task in tasks:
        task_id = task.get("task_id")
        if not task_id:
            continue
        task_ids.add(task_id)
        depends_on = task.get("depends_on", [])
        graph[task_id] = depends_on

    # DFS to detect cycles
    visited: Set[str] = set()
    rec_stack: Set[str] = set()
    cycle_path: List[str] = []

    def has_cycle(node: str, path: List[str]) -> bool:
        visited.add(node)
        rec_stack.add(node)
        path.append(node)

        for dep in graph.get(node, []):
            if dep not in task_ids:
                # Dependency doesn't exist - not a cycle, but an error
                continue
            if dep not in visited:
                if has_cycle(dep, path):
                    return True
            elif dep in rec_stack:
                # Found a cycle
                cycle_path.extend(path[path.index(dep):])
                cycle_path.append(dep)
                return True

        rec_stack.remove(node)
        path.pop()
        return False

    # Check all nodes
    for task_id in task_ids:
        if task_id not in visited:
            if has_cycle(task_id, []):
                return (True, cycle_path)

    return (False, [])


def get_ready_tasks(tasks: List[Dict[str, Any]], completed_task_ids: Set[str]) -> List[Dict[str, Any]]:
    """
    Get tasks that are ready to execute (all dependencies satisfied).

    Args:
        tasks: List of all tasks
        completed_task_ids: Set of task_ids that have been completed

    Returns:
        List of tasks ready to execute
    """
    ready = []

    for task in tasks:
        task_id = task.get("task_id")
        depends_on = set(task.get("depends_on", []))

        # Task is ready if all dependencies are completed
        if depends_on.issubset(completed_task_ids):
            ready.append(task)

    return ready


def resolve_task_dependencies(tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Resolve task dependencies and return tasks in execution order.

    This function:
    1. Detects circular dependencies
    2. Orders tasks by their dependencies
    3. Returns tasks ready to execute (those with no dependencies or all deps satisfied)

    Args:
        tasks: List of task dictionaries

    Returns:
        List of tasks in execution order (ready to execute first)
    """
    # First, check for circular dependencies
    has_cycle, cycle_path = detect_circular_dependencies(tasks)
    if has_cycle:
        raise ValueError(f"Circular dependency detected: {' -> '.join(cycle_path)}")

    # Order tasks by their order field
    ordered_tasks = order_tasks_by_dependency(tasks)

    return ordered_tasks



# Task Ordering Node

In [ ]:
def task_ordering_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Task Ordering Node: Order tasks and resolve dependencies.

    This node:
    1. Orders tasks by their dependencies
    2. Detects circular dependencies
    3. Creates initial task queue with ready-to-execute tasks

    Input:
        - mission_tasks (List[Dict]): All tasks for the mission

    Output:
        - task_queue (List[Dict]): Tasks ready to execute (ordered)
        - tasks_total (int): Total number of tasks
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    mission_tasks = state.get("mission_tasks", [])

    if not mission_tasks:
        return {
            "errors": errors + ["task_ordering_node: mission_tasks is required"]
        }

    try:
        # Resolve dependencies and order tasks
        ordered_tasks = resolve_task_dependencies(mission_tasks)

        # Initial task queue: tasks with no dependencies (ready to execute)
        completed_task_ids = set()  # No tasks completed yet
        ready_tasks = get_ready_tasks(ordered_tasks, completed_task_ids)

        # Initialize mission start time if not set
        mission_start_time = state.get("mission_start_time")
        if not mission_start_time:
            mission_start_time = datetime.now().isoformat()

        return {
            "task_queue": ready_tasks,
            "tasks_total": len(mission_tasks),
            "tasks_completed": 0,
            "executed_tasks": [],
            "task_results": {},
            "mission_start_time": mission_start_time,
            "mission_status": "in_progress",
            "errors": errors
        }
    except ValueError as e:
        # Circular dependency or other validation error
        return {
            "errors": errors + [f"task_ordering_node: {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"task_ordering_node: Unexpected error: {str(e)}"]
        }


def task_execution_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Task Execution Node: Execute the next task in the queue.

    This node:
    1. Gets next task from queue
    2. Selects appropriate agent
    3. Executes task via agent
    4. Updates executed tasks and results
    5. Updates task queue with newly ready tasks

    Input:
        - task_queue (List[Dict]): Tasks ready to execute
        - executed_tasks (List[Dict]): Already completed tasks
        - specialized_agents (List[Dict]): Available agents
        - capabilities_matrix (List[Dict]): Task-to-agent mappings
        - mission_tasks (List[Dict]): All tasks for the mission

    Output:
        - executed_tasks (List[Dict]): Updated with new execution result
        - task_results (Dict): Updated with new result
        - task_queue (List[Dict]): Updated queue (removed executed, added newly ready)
        - current_task (Optional[Dict]): Currently executing task (or None if done)
        - tasks_completed (int): Updated count
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    task_queue = state.get("task_queue", [])
    executed_tasks = state.get("executed_tasks", [])
    task_results = state.get("task_results", {})
    specialized_agents = state.get("specialized_agents", [])
    capabilities_matrix = state.get("capabilities_matrix", [])
    mission_tasks = state.get("mission_tasks", [])
    config = MissionOrchestratorConfig()

    # Check if there are tasks to execute
    if not task_queue:
        # No more tasks to execute
        return {
            "current_task": None,
            "errors": errors
        }

    try:
        # Get next task from queue
        current_task = task_queue[0]
        task_id = current_task.get("task_id")

        # Select agent for this task
        agent = select_agent_for_task(
            task_id,
            capabilities_matrix,
            specialized_agents,
            strategy=config.agent_selection_strategy
        )

        if not agent:
            return {
                "errors": errors + [f"task_execution_node: No capable agent found for task {task_id}"]
            }

        # Execute task
        execution_result = execute_task(current_task, agent, mock_execution=True)

        # Update executed tasks
        executed_tasks = executed_tasks + [execution_result]

        # Update task results lookup
        task_results[task_id] = execution_result

        # Remove executed task from queue
        remaining_queue = task_queue[1:]

        # Get newly ready tasks (tasks whose dependencies are now satisfied)
        completed_task_ids = {t.get("task_id") for t in executed_tasks if t.get("status") == "completed"}
        newly_ready_tasks = get_ready_tasks(mission_tasks, completed_task_ids)

        # Filter out already executed tasks and tasks already in queue
        executed_task_ids = {t.get("task_id") for t in executed_tasks}
        queue_task_ids = {t.get("task_id") for t in remaining_queue}

        # Add newly ready tasks that aren't already executed or in queue
        for task in newly_ready_tasks:
            task_id_new = task.get("task_id")
            if task_id_new not in executed_task_ids and task_id_new not in queue_task_ids:
                remaining_queue.append(task)

        # Sort remaining queue by order
        remaining_queue = sorted(remaining_queue, key=lambda t: t.get("order", 999))

        # Update tasks completed count
        tasks_completed = len([t for t in executed_tasks if t.get("status") == "completed"])

        return {
            "executed_tasks": executed_tasks,
            "task_results": task_results,
            "task_queue": remaining_queue,
            "current_task": current_task if remaining_queue else None,  # None if queue is empty
            "tasks_completed": tasks_completed,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"task_execution_node: Unexpected error: {str(e)}"]
        }


# Standalone test script for task execution utilities and nodes

In [ ]:
"""Standalone test script for task execution utilities and nodes"""

from agents.mission_orchestrator.utilities.task_ordering import (
    order_tasks_by_dependency,
    detect_circular_dependencies,
    get_ready_tasks,
    resolve_task_dependencies
)
from agents.mission_orchestrator.utilities.agent_selection import (
    select_agent_for_task,
    get_capable_agents
)
from agents.mission_orchestrator.utilities.task_execution import (
    execute_task,
    mock_agent_execution
)
from agents.mission_orchestrator.nodes import task_ordering_node, task_execution_node
from agents.mission_orchestrator.utilities.data_loading import (
    load_mission_tasks,
    load_specialized_agents,
    load_agent_capabilities_matrix
)
from config import MissionOrchestratorState


def test_task_ordering_utilities():
    """Test task ordering utilities"""
    print("=" * 60)
    print("Testing Task Ordering Utilities")
    print("=" * 60)

    # Load sample tasks
    tasks = load_mission_tasks("M001")

    # Test 1: Order tasks by dependency
    print("\n1. Ordering tasks by dependency...")
    ordered = order_tasks_by_dependency(tasks)
    print(f"   ✓ Ordered {len(ordered)} tasks")
    for task in ordered:
        print(f"   - {task['task_id']}: order={task['order']}, depends_on={task['depends_on']}")

    # Test 2: Detect circular dependencies (should not find any)
    print("\n2. Detecting circular dependencies...")
    has_cycle, cycle_path = detect_circular_dependencies(tasks)
    if has_cycle:
        print(f"   ✗ Found circular dependency: {' -> '.join(cycle_path)}")
    else:
        print("   ✓ No circular dependencies found")

    # Test 3: Get ready tasks
    print("\n3. Getting ready tasks (no dependencies)...")
    completed = set()
    ready = get_ready_tasks(tasks, completed)
    print(f"   ✓ Found {len(ready)} ready tasks")
    for task in ready:
        print(f"   - {task['task_id']}: {task['task']}")

    # Test 4: Get ready tasks after T1 completed
    print("\n4. Getting ready tasks after T1 completed...")
    completed = {"T1"}
    ready = get_ready_tasks(tasks, completed)
    print(f"   ✓ Found {len(ready)} ready tasks")
    for task in ready:
        print(f"   - {task['task_id']}: {task['task']}")

    # Test 5: Resolve task dependencies
    print("\n5. Resolving task dependencies...")
    try:
        resolved = resolve_task_dependencies(tasks)
        print(f"   ✓ Resolved {len(resolved)} tasks")
        for task in resolved:
            print(f"   - {task['task_id']}: order={task['order']}")
    except ValueError as e:
        print(f"   ✗ Error: {e}")

    print("\n" + "=" * 60)
    print("Task Ordering Utilities Tests Complete!")
    print("=" * 60)


def test_agent_selection_utilities():
    """Test agent selection utilities"""
    print("\n" + "=" * 60)
    print("Testing Agent Selection Utilities")
    print("=" * 60)

    agents = load_specialized_agents()
    matrix = load_agent_capabilities_matrix()

    # Test 1: Select agent for task
    print("\n1. Selecting agent for task T1...")
    agent = select_agent_for_task("T1", matrix, agents)
    if agent:
        print(f"   ✓ Selected: {agent['name']} ({agent['agent_id']})")
    else:
        print("   ✗ No agent selected")

    # Test 2: Get capable agents
    print("\n2. Getting all capable agents for task T1...")
    capable = get_capable_agents("T1", matrix, agents)
    print(f"   ✓ Found {len(capable)} capable agents")
    for agent in capable:
        print(f"   - {agent['name']} ({agent['agent_id']})")

    # Test 3: Select agent for task T2
    print("\n3. Selecting agent for task T2...")
    agent = select_agent_for_task("T2", matrix, agents)
    if agent:
        print(f"   ✓ Selected: {agent['name']} ({agent['agent_id']})")

    print("\n" + "=" * 60)
    print("Agent Selection Utilities Tests Complete!")
    print("=" * 60)


def test_task_execution_utilities():
    """Test task execution utilities"""
    print("\n" + "=" * 60)
    print("Testing Task Execution Utilities")
    print("=" * 60)

    agents = load_specialized_agents()
    tasks = load_mission_tasks("M001")
    matrix = load_agent_capabilities_matrix()

    # Test 1: Mock agent execution
    print("\n1. Testing mock agent execution...")
    result = mock_agent_execution("A1", "Collect customer information")
    print(f"   ✓ Mock execution completed")
    print(f"   Status: {result['status']}")
    print(f"   Output: {result['output']}")

    # Test 2: Execute task
    print("\n2. Executing task T1...")
    task = tasks[0]  # T1
    agent = select_agent_for_task("T1", matrix, agents)
    if agent:
        execution_result = execute_task(task, agent, mock_execution=True)
        print(f"   ✓ Task executed")
        print(f"   Task ID: {execution_result['task_id']}")
        print(f"   Agent: {execution_result['agent_name']}")
        print(f"   Status: {execution_result['status']}")
        print(f"   Duration: {execution_result['duration_minutes']:.2f} minutes")
        if execution_result.get('error'):
            print(f"   Error: {execution_result['error']}")

    print("\n" + "=" * 60)
    print("Task Execution Utilities Tests Complete!")
    print("=" * 60)


def test_task_ordering_node():
    """Test task ordering node"""
    print("\n" + "=" * 60)
    print("Testing Task Ordering Node")
    print("=" * 60)

    # Setup state with mission tasks
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "mission_tasks": load_mission_tasks("M001"),
        "errors": []
    }

    result = task_ordering_node(state)

    if "task_queue" in result:
        print("   ✓ Node executed successfully")
        print(f"   - Total tasks: {result['tasks_total']}")
        print(f"   - Tasks in queue: {len(result['task_queue'])}")
        print(f"   - Tasks completed: {result['tasks_completed']}")
        print(f"   - Mission status: {result['mission_status']}")
        print("\n   Ready tasks:")
        for task in result['task_queue']:
            print(f"     - {task['task_id']}: {task['task']}")
    else:
        print("   ✗ Node failed")
        if result.get("errors"):
            for error in result["errors"]:
                print(f"     - {error}")

    print("\n" + "=" * 60)
    print("Task Ordering Node Test Complete!")
    print("=" * 60)


def test_task_execution_node():
    """Test task execution node"""
    print("\n" + "=" * 60)
    print("Testing Task Execution Node")
    print("=" * 60)

    # Setup state with all required data
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "mission_tasks": load_mission_tasks("M001"),
        "specialized_agents": load_specialized_agents(),
        "capabilities_matrix": load_agent_capabilities_matrix(),
        "task_queue": [],  # Will be set by task_ordering_node
        "executed_tasks": [],
        "task_results": {},
        "errors": []
    }

    # First, order tasks
    ordering_result = task_ordering_node(state)
    state = {**state, **ordering_result}

    print(f"\nInitial state:")
    print(f"  - Tasks in queue: {len(state['task_queue'])}")
    print(f"  - Tasks completed: {state['tasks_completed']}")

    # Execute tasks one by one
    max_iterations = 10  # Safety limit
    iteration = 0

    while state.get("task_queue") and iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        print(f"Tasks in queue: {len(state['task_queue'])}")

        if state['task_queue']:
            next_task = state['task_queue'][0]
            print(f"Executing: {next_task['task_id']} - {next_task['task']}")

        execution_result = task_execution_node(state)
        state = {**state, **execution_result}

        if execution_result.get("current_task") is None and not state.get("task_queue"):
            print("   ✓ All tasks completed!")
            break

        if execution_result.get("executed_tasks"):
            last_executed = execution_result["executed_tasks"][-1]
            print(f"   ✓ Task {last_executed['task_id']} completed")
            print(f"     Agent: {last_executed['agent_name']}")
            print(f"     Status: {last_executed['status']}")
            print(f"     Duration: {last_executed['duration_minutes']:.2f} min")

        if execution_result.get("errors"):
            print(f"   ⚠ Errors: {execution_result['errors']}")

    print(f"\n--- Final State ---")
    print(f"  - Tasks completed: {state['tasks_completed']}/{state['tasks_total']}")
    print(f"  - Tasks in queue: {len(state.get('task_queue', []))}")
    print(f"  - Executed tasks: {len(state.get('executed_tasks', []))}")

    if state.get("executed_tasks"):
        print("\n  Execution Summary:")
        for task in state['executed_tasks']:
            print(f"    - {task['task_id']}: {task['status']} ({task['duration_minutes']:.2f} min)")

    print("\n" + "=" * 60)
    print("Task Execution Node Test Complete!")
    print("=" * 60)


def test_full_flow():
    """Test full flow: goal → planning → data loading → task ordering → task execution"""
    print("\n" + "=" * 60)
    print("Testing Full Flow: Goal → Planning → Data Loading → Task Ordering → Task Execution")
    print("=" * 60)

    from agents.mission_orchestrator.nodes import goal_node, planning_node, data_loading_node

    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Step 1: Goal node
    print("\n1. Executing goal node...")
    goal_result = goal_node(state)
    state = {**state, **goal_result}
    print(f"   ✓ Goal defined")

    # Step 2: Planning node
    print("\n2. Executing planning node...")
    planning_result = planning_node(state)
    state = {**state, **planning_result}
    print(f"   ✓ Plan created")

    # Step 3: Data loading node
    print("\n3. Executing data loading node...")
    data_result = data_loading_node(state)
    state = {**state, **data_result}
    print(f"   ✓ Data loaded")

    # Step 4: Task ordering node
    print("\n4. Executing task ordering node...")
    ordering_result = task_ordering_node(state)
    state = {**state, **ordering_result}
    print(f"   ✓ Tasks ordered: {len(state['task_queue'])} ready to execute")

    # Step 5: Execute all tasks
    print("\n5. Executing tasks...")
    max_iterations = 10
    iteration = 0

    while state.get("task_queue") and iteration < max_iterations:
        iteration += 1
        next_task = state['task_queue'][0] if state['task_queue'] else None
        if next_task:
            print(f"   Executing {next_task['task_id']}: {next_task['task']}...")

        execution_result = task_execution_node(state)
        state = {**state, **execution_result}

        if execution_result.get("executed_tasks"):
            last_executed = execution_result["executed_tasks"][-1]
            print(f"     ✓ Completed by {last_executed['agent_name']}")

        if not state.get("task_queue"):
            print(f"\n   ✓ All {state['tasks_completed']} tasks completed!")
            break

    print(f"\n--- Mission Execution Summary ---")
    print(f"  Mission: {state['mission']['mission_name']}")
    print(f"  Tasks: {state['tasks_completed']}/{state['tasks_total']} completed")
    print(f"  Status: {state['mission_status']}")

    if state.get("executed_tasks"):
        total_duration = sum(t.get('duration_minutes', 0) for t in state['executed_tasks'])
        print(f"  Total duration: {total_duration:.2f} minutes")

    print("\n" + "=" * 60)
    print("Full Flow Test Complete!")
    print("=" * 60)


if __name__ == "__main__":
    try:
        test_task_ordering_utilities()
        test_agent_selection_utilities()
        test_task_execution_utilities()
        test_task_ordering_node()
        test_task_execution_node()
        test_full_flow()
        print("\n✅ All tests completed successfully!")
    except Exception as e:
        print(f"\n❌ Error during testing: {e}")
        import traceback
        traceback.print_exc()



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python test_task_execution_standalone.py
============================================================
Testing Task Ordering Utilities
============================================================

1. Ordering tasks by dependency...
   ✓ Ordered 3 tasks
   - T1: order=1, depends_on=[]
   - T2: order=2, depends_on=['T1']
   - T3: order=3, depends_on=['T2']

2. Detecting circular dependencies...
   ✓ No circular dependencies found

3. Getting ready tasks (no dependencies)...
   ✓ Found 1 ready tasks
   - T1: Collect customer information

4. Getting ready tasks after T1 completed...
   ✓ Found 2 ready tasks
   - T1: Collect customer information
   - T2: Verify documents

5. Resolving task dependencies...
   ✓ Resolved 3 tasks
   - T1: order=1
   - T2: order=2
   - T3: order=3

============================================================
Task Ordering Utilities Tests Complete!
============================================================

============================================================
Testing Agent Selection Utilities
============================================================

1. Selecting agent for task T1...
   ✓ Selected: Data Collection Agent (A1)

2. Getting all capable agents for task T1...
   ✓ Found 1 capable agents
   - Data Collection Agent (A1)

3. Selecting agent for task T2...
   ✓ Selected: Document Verification Agent (A2)

============================================================
Agent Selection Utilities Tests Complete!
============================================================

============================================================
Testing Task Execution Utilities
============================================================

1. Testing mock agent execution...
   ✓ Mock execution completed
   Status: completed
   Output: Mock execution result for Collect customer information by agent A1

2. Executing task T1...
   ✓ Task executed
   Task ID: T1
   Agent: Data Collection Agent
   Status: completed
   Duration: 0.00 minutes

============================================================
Task Execution Utilities Tests Complete!
============================================================

============================================================
Testing Task Ordering Node
============================================================
   ✓ Node executed successfully
   - Total tasks: 3
   - Tasks in queue: 1
   - Tasks completed: 0
   - Mission status: in_progress

   Ready tasks:
     - T1: Collect customer information

============================================================
Task Ordering Node Test Complete!
============================================================

============================================================
Testing Task Execution Node
============================================================

Initial state:
  - Tasks in queue: 1
  - Tasks completed: 0

--- Iteration 1 ---
Tasks in queue: 1
Executing: T1 - Collect customer information
   ✓ Task T1 completed
     Agent: Data Collection Agent
     Status: completed
     Duration: 0.00 min

--- Iteration 2 ---
Tasks in queue: 1
Executing: T2 - Verify documents
   ✓ Task T2 completed
     Agent: Document Verification Agent
     Status: completed
     Duration: 0.00 min

--- Iteration 3 ---
Tasks in queue: 1
Executing: T3 - Schedule onboarding call
   ✓ All tasks completed!

--- Final State ---
  - Tasks completed: 3/3
  - Tasks in queue: 0
  - Executed tasks: 3

  Execution Summary:
    - T1: completed (0.00 min)
    - T2: completed (0.00 min)
    - T3: completed (0.00 min)

============================================================
Task Execution Node Test Complete!
============================================================

============================================================
Testing Full Flow: Goal → Planning → Data Loading → Task Ordering → Task Execution
============================================================

1. Executing goal node...
   ✓ Goal defined

2. Executing planning node...
   ✓ Plan created

3. Executing data loading node...
   ✓ Data loaded

4. Executing task ordering node...
   ✓ Tasks ordered: 1 ready to execute

5. Executing tasks...
   Executing T1: Collect customer information...
     ✓ Completed by Data Collection Agent
   Executing T2: Verify documents...
     ✓ Completed by Document Verification Agent
   Executing T3: Schedule onboarding call...
     ✓ Completed by Scheduling Agent

   ✓ All 3 tasks completed!

--- Mission Execution Summary ---
  Mission: Reduce Customer Onboarding Time
  Tasks: 3/3 completed
  Status: in_progress
  Total duration: 0.01 minutes

============================================================
Full Flow Test Complete!
============================================================

